In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('pbp_test.csv',index_col=0)

def str_to_tuple(s):
    return tuple(s.strip('() ').replace(' ', '').split(','))

df['h_lineup'] = df.h_lineup.apply(lambda x: str_to_tuple(x))
df['a_lineup'] = df.a_lineup.apply(lambda x: str_to_tuple(x))

eventdict = {1:'FGM',
             2:'FGA',
             3:'FTA',
             4:'TRB',
             5:'TURNOVER',
             6:'FOUL',
             8:'SUB',
             9:'TIMEOUT',
             10:'JUMP_BALL',
             12:'Q_START',
             13:'Q_END',
             18:'REPLAY'             
            }
cols = ['game_id', 'lineup','player_id','PTS','FGM', 'FGA', '3PM', '3PA', 'FTM', 'FTA', 'REB', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'MIN', 'POSS']


In [2]:
#allll this ugly stuff just to get a column called 'rebound_type'
mask = (df.eventmsgtype == 4) & (~df.homedescription.isnull() | ~df.visitordescription.isnull())
pattern = r'Off:(\d+) Def:(\d+)'
#getting rebound counts from play description and filling nulls, and casting to int
df[['offensive_rebounds', 'defensive_rebounds']] = df['homedescription'].str.extract(pattern)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df['visitordescription'].str.extract(pattern)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].fillna(0)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].astype(int)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df[['offensive_rebounds_v', 'defensive_rebounds_v']].fillna(0)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df[['offensive_rebounds_v', 'defensive_rebounds_v']].astype(int)

df.sort_values(by=['player1_id','period','total_elapsed_time','eventnum'], inplace=True)

#comparing reb total to previous to check what type of rebound occurred
df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df['rebound_type'] = ''
df.loc[(df.defensive_rebounds > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.loc[(df.defensive_rebounds_v > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds_v > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.drop([ 'offensive_rebounds', 'defensive_rebounds',
       'offensive_rebounds_v', 'defensive_rebounds_v', 'def_reb_previous',
       'off_reb_previous'],inplace=True,axis=1)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_15468\3264647376.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_15468\3264647376.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)


In [3]:
fga_filter = (df.eventmsgtype==1) | (df.eventmsgtype==2)
two_pt_filter = (df.eventmsgtype==1) & ~((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_pt_filter = (df.eventmsgtype==1) & ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_a_filter = ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
ft_filter = (df.eventmsgtype==3) & ((df.homedescription.str.contains('PTS',na=False)) | (df.visitordescription.str.contains('PTS',na=False)))
fta_filter = df.eventmsgtype==3 
assist_filter = (df.eventmsgtype==1) & ~df.player2_name.isnull()

orb_filter = df.rebound_type == 'ORB'
drb_filter = df.rebound_type == 'DRB'
trb_filter = (df.rebound_type == 'DRB') | (df.rebound_type == 'ORB')
stl_filter = df.homedescription.str.contains('STEAL',na=False) | df.visitordescription.str.contains('STEAL',na=False)
blk_filter = df.homedescription.str.contains('BLOCK',na=False) | df.visitordescription.str.contains('BLOCK',na=False)
to_filter = df.eventmsgtype==5
foul_filter = df.eventmsgtype==6

In [38]:
visitor_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.homedescription.isna())) | # home team drb means visitor poss ends
(df.homedescription.str.contains('STEAL',na=False)) |# home team steal means visitor poss ends
((df.eventmsgtype==1) & ~(df.visitordescription.isna())) |# visitor team makes shot means visitor poss ends
((df.eventmsgtype==3) & df.visitordescription.str.contains('1 of')) #visitor team takes ft means poss ends
) 

home_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.visitordescription.isna())) | # home team drb means visitor poss ends
(df.visitordescription.str.contains('STEAL',na=False)) |# home team steal means visitor poss ends
((df.eventmsgtype==1) & ~(df.homedescription.isna())) |# visitor team makes shot means visitor poss ends
((df.eventmsgtype==3) & df.homedescription.str.contains('1 of')) #visitor team takes ft means poss ends
) 


df.loc[visitor_poss_filter][['eventmsgtype','homedescription','visitordescription','player1_id','player1_name','player2_id','player2_name','player3_id','player3_name']]

#
#
#
#            change this so that it only counts made free throws? that are the last in the series

,eventmsgtype,homedescription,visitordescription,player1_id,player1_name,player2_id,player2_name,player3_id,player3_name
2,1,NaN,Thompson 25' 3PT Jump Shot (3 PTS) (Curry 1 AST),202691,Klay Thompson,201939,Stephen Curry,0,NaN
4,1,NaN,Thompson 25' 3PT Jump Shot (6 PTS) (Curry 2 AST),202691,Klay Thompson,201939,Stephen Curry,0,NaN
8,4,Sharpe REBOUND (Off:0 Def:1),NaN,1631101,Shaedon Sharpe,0,NaN,0,NaN
11,1,NaN,Thompson 1' Running Reverse Layup (8 PTS) (DiV...,202691,Klay Thompson,1628978,Donte DiVincenzo,0,NaN
15,3,NaN,DiVincenzo Free Throw 1 of 2 (1 PTS),1628978,Donte DiVincenzo,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...
420,1,NaN,Baldwin Jr. 25' 3PT Running Jump Shot (8 PTS) ...,1631116,Patrick Baldwin,1630228,Jonathan Kuminga,0,NaN
424,1,NaN,Quinones 32' 3PT Jump Shot (9 PTS) (Lamb 2 AST),1631311,Lester Quinones,1630237,Anthony Lamb,0,NaN
435,1,NaN,Moody 2' Cutting Dunk Shot (23 PTS) (Lamb 3 AST),1630541,Moses Moody,1630237,Anthony Lamb,0,NaN
437,1,NaN,Moody 1' Cutting Dunk Shot (25 PTS) (Kuminga 6...,1630541,Moses Moody,1630228,Jonathan Kuminga,0,NaN


In [39]:
df.columns

Index(['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
       'pctimestring', 'homedescription', 'neutraldescription',
       'visitordescription', 'score', 'scoremargin', 'player1_id',
       'player1_name', 'player1_team_abbreviation', 'player2_id',
       'player2_name', 'player2_team_abbreviation', 'player3_id',
       'player3_name', 'player3_team_abbreviation', 'timeinseconds',
       'play_elapsed_time', 'total_elapsed_time', 'max_time', 'time_remaining',
       'h_lineup', 'a_lineup', 'rebound_type'],
      dtype='object')

In [42]:
def get_stat_df(pbp_df,filter,stat_type,stat_value,player_num='1'):
    filtered_df = pbp_df.loc[filter][['game_id','eventnum',f'player{player_num}_id',f'player{player_num}_name','h_lineup']]
    filtered_df['stat'] = stat_type
    filtered_df['value'] = stat_value
    return filtered_df.values.tolist()

#results is a list of every stat credit for every play
results = get_stat_df(df,two_pt_filter,'PTS',2)
results += get_stat_df(df,three_pt_filter,'PTS',3)
results += get_stat_df(df,ft_filter,'PTS',1)

results += get_stat_df(df,fga_filter,'FGA',1)
results += get_stat_df(df,fta_filter,'FTA',1)
results += get_stat_df(df,three_a_filter,'3PA',1)

results += get_stat_df(df,trb_filter,'TRB',1)
results += get_stat_df(df,drb_filter,'DRB',1)
results += get_stat_df(df,orb_filter,'ORB',1)

results += get_stat_df(df,stl_filter,'STL',1,player_num=2)
results += get_stat_df(df,blk_filter,'BLK',1,player_num=3)
results += get_stat_df(df,to_filter,'TOV',1,player_num=1)
results += get_stat_df(df,foul_filter,'PF',1)

results += get_stat_df(df,assist_filter,'AST',1,player_num='2')
results[-1]

[41800237,
 654,
 1627750,
 'Jamal Murray',
 ('200794', '203914', '203999', '1627750', '1628470'),
 'AST',
 1]

In [43]:
play_credit_df = pd.DataFrame(results,columns=['game_id','eventnum','player_id','player_name','lineup','stat_type','stat_value'])
play_credit_df.groupby(by=['player_id', 'stat_type','player_name'])['stat_value'].sum()

player_id  stat_type  player_name 
200794     3PA        Paul Millsap     2
           AST        Paul Millsap     1
           BLK        Paul Millsap     3
           DRB        Paul Millsap     6
           FGA        Paul Millsap    13
                                      ..
1628470    ORB        Torrey Craig     4
           PF         Torrey Craig     2
           PTS        Torrey Craig     8
           TOV        Torrey Craig     1
           TRB        Torrey Craig     8
Name: stat_value, Length: 159, dtype: int64

In [44]:
game_play_agg = play_credit_df.groupby(by=['lineup','player_id', 'stat_type','player_name'],as_index=False)['stat_value'].sum()
game_play_agg

,lineup,player_id,stat_type,player_name,stat_value
0,"(200794, 203115, 203486, 1627736, 1628420)",200794,DRB,Paul Millsap,2
1,"(200794, 203115, 203486, 1627736, 1628420)",200794,FGA,Paul Millsap,4
2,"(200794, 203115, 203486, 1627736, 1628420)",200794,FTA,Paul Millsap,2
3,"(200794, 203115, 203486, 1627736, 1628420)",200794,ORB,Paul Millsap,1
4,"(200794, 203115, 203486, 1627736, 1628420)",200794,PTS,Paul Millsap,1
...,...,...,...,...,...
370,"(203486, 203914, 203999, 1627750, 1628470)",1627750,ORB,Jamal Murray,1
371,"(203486, 203914, 203999, 1627750, 1628470)",1627750,PTS,Jamal Murray,2
372,"(203486, 203914, 203999, 1627750, 1628470)",1627750,TRB,Jamal Murray,2
373,"(203486, 203914, 203999, 1627750, 1628470)",1628470,3PA,Torrey Craig,1


In [45]:
pivoted_game = game_play_agg.pivot(index=['lineup', 'player_id', 'player_name'], columns='stat_type', values='stat_value').fillna(0).reset_index()
pivoted_game

stat_type,lineup,player_id,player_name,3PA,AST,BLK,DRB,FGA,FTA,ORB,PF,PTS,STL,TOV,TRB
0,"(200794, 203115, 203486, 1627736, 1628420)",200794,Paul Millsap,0.0,0.0,0.0,2.0,4.0,2.0,1.0,0.0,1.0,0.0,0.0,3.0
1,"(200794, 203115, 203486, 1627736, 1628420)",202323,Evan Turner,0.0,0.0,1.0,3.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,3.0
2,"(200794, 203115, 203486, 1627736, 1628420)",203081,Damian Lillard,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"(200794, 203115, 203486, 1627736, 1628420)",203115,Will Barton,1.0,0.0,0.0,1.0,6.0,0.0,1.0,1.0,4.0,0.0,0.0,2.0
4,"(200794, 203115, 203486, 1627736, 1628420)",203468,CJ McCollum,0.0,1.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"(203486, 203914, 203999, 1627750, 1628470)",203486,Mason Plumlee,0.0,0.0,1.0,2.0,0.0,2.0,0.0,2.0,1.0,0.0,0.0,2.0
94,"(203486, 203914, 203999, 1627750, 1628470)",203918,Rodney Hood,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
95,"(203486, 203914, 203999, 1627750, 1628470)",203999,Nikola Jokic,0.0,0.0,0.0,1.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
96,"(203486, 203914, 203999, 1627750, 1628470)",1627750,Jamal Murray,1.0,1.0,0.0,1.0,5.0,0.0,1.0,0.0,2.0,0.0,0.0,2.0


In [46]:
pivoted_game.groupby(['player_id','player_name'],as_index=False).sum()

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_17608\3685938496.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pivoted_game.groupby(['player_id','player_name'],as_index=False).sum()


stat_type,player_id,player_name,3PA,AST,BLK,DRB,FGA,FTA,ORB,PF,PTS,STL,TOV,TRB
0,200794,Paul Millsap,2.0,1.0,3.0,6.0,13.0,6.0,1.0,6.0,10.0,0.0,0.0,7.0
1,202323,Evan Turner,0.0,2.0,1.0,3.0,7.0,9.0,4.0,4.0,14.0,0.0,0.0,7.0
2,202329,Al-Farouq Aminu,2.0,0.0,0.0,3.0,4.0,2.0,0.0,1.0,3.0,0.0,1.0,3.0
3,202683,Enes Freedom,1.0,1.0,0.0,0.0,13.0,0.0,12.0,3.0,12.0,0.0,1.0,12.0
4,203081,Damian Lillard,9.0,8.0,0.0,10.0,17.0,6.0,0.0,3.0,13.0,3.0,1.0,10.0
5,203086,Meyers Leonard,2.0,0.0,0.0,0.0,4.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0
6,203090,Maurice Harkless,1.0,3.0,1.0,1.0,5.0,1.0,4.0,5.0,6.0,1.0,0.0,5.0
7,203115,Will Barton,2.0,1.0,0.0,2.0,9.0,0.0,1.0,3.0,8.0,0.0,0.0,3.0
8,203468,CJ McCollum,3.0,1.0,1.0,4.0,29.0,2.0,5.0,1.0,37.0,1.0,0.0,9.0
9,203486,Mason Plumlee,0.0,0.0,2.0,5.0,3.0,5.0,1.0,3.0,4.0,0.0,0.0,6.0


In [86]:
# import required module
from pathlib import Path
 
# assign directory
directory = 'pbp_raw'
file_names = []
# iterate over files in
# that directory
files = Path(directory).glob('*')
for file in files:
    file_names.append(file)

In [107]:
[str(file) for file in file_names]

['\\0022200002_pbp.csv',
 '\\0022200026_pbp.csv',
 '\\0022200042_pbp.csv',
 '\\0022200055_pbp.csv',
 '\\0022200068_pbp.csv',
 '\\0022200082_pbp.csv',
 '\\0022200091_pbp.csv',
 '\\0022200104_pbp.csv',
 '\\0022200118_pbp.csv',
 '\\0022200128_pbp.csv',
 '\\0022200233_pbp.csv',
 '\\0022200245_pbp.csv',
 '\\0022200253_pbp.csv',
 '\\0022200271_pbp.csv',
 '\\0022200284_pbp.csv',
 '\\0022200291_pbp.csv',
 '\\0022200310_pbp.csv',
 '\\0022200336_pbp.csv',
 '\\0022200342_pbp.csv']

In [108]:
import re

In [113]:
pattern = r'\d+'
[re.findall(pattern, str(file)) for file in file_names]

[['0022200002'],
 ['0022200026'],
 ['0022200042'],
 ['0022200055'],
 ['0022200068'],
 ['0022200082'],
 ['0022200091'],
 ['0022200104'],
 ['0022200118'],
 ['0022200128'],
 ['0022200233'],
 ['0022200245'],
 ['0022200253'],
 ['0022200271'],
 ['0022200284'],
 ['0022200291'],
 ['0022200310'],
 ['0022200336'],
 ['0022200342']]

In [116]:
import os.path

path = 'pbp_raw/0022200026_pbp.csv'

check_file = os.path.isfile(path)

print(check_file)

True


In [111]:
numbers = re.findall(r'\d+', str(file_names[0]))

In [112]:
numbers

['0022200002']